# QPQ9 Sample

In [1]:
%matplotlib inline

In [2]:
# imports
from scipy.io import readsav as scirs
import copy, os, glob, imp
import numpy as np

from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.table import Column, Table
from astropy.io import fits, ascii

from xastropy.xutils import fits as xxf
from xastropy.xutils import xdebug as xdb

from specdb.specdb import IgmSpec, SpecDB
from specdb import query_catalog as spqcat
from specdb import interface_group as spigp
from specdb import utils as spdbu

from linetools import utils as ltu

from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0=0.26)

import enigma

## QPQ Sample Code
Modifying lya_qpq_sample.pro in pro/data/photo

Created QPQ-CGM-IR  [Joe did]

QPQ-CGM-IR_00.000-00.300_Sat-Apr-25-21:33:45-2015.sav

### Try MgII $z_{\rm em}$ someday

## Read the Sample and Collate

In [236]:
# IDL savefile read failed..
#dat = scirs(os.getenv('DROPBOX_DIR')+'/QSOPairs/QSO-CGM-IR/QPQ-CGM-IR_00.000-00.300_Sat-Apr-25-21:33:45-2015.sav',verbose=True)
#dat

# Wrote a binary FITS table from IDL
#bgfil = 'QPQ-CGM-IR_00.000-00.300_Sat-Apr-25-21:33:45-2015_qsobg.fits.gz'
#fgfil = 'QPQ-CGM-IR_00.000-00.300_Sat-Apr-25-21:33:45-2015_qsofg.fits.gz'
#
bgfil = 'QPQ-CGM-IR_00.000-00.300_Tue-May-26-18:12:57-2015_qsobg.fits'
fgfil = 'QPQ-CGM-IR_00.000-00.300_Tue-May-26-18:12:57-2015_qsofg.fits'
#
qso_bg = xxf.bintab_to_table(os.getenv('DROPBOX_DIR')+'/QSOPairs/QPQ-CGM-IR/'+bgfil)
#
qso_fg = xxf.bintab_to_table(os.getenv('DROPBOX_DIR')+'/QSOPairs/QPQ-CGM-IR/'+fgfil)
#qso_fg[['NAME','LYA_FILE', 'Z_IR']]

x.fits.bintab_to_table: Reading /Users/lwymarie/Dropbox//QSOPairs/QPQ-CGM-IR/QPQ-CGM-IR_00.000-00.300_Tue-May-26-18:12:57-2015_qsobg.fits
x.fits.bintab_to_table: Reading /Users/lwymarie/Dropbox//QSOPairs/QPQ-CGM-IR/QPQ-CGM-IR_00.000-00.300_Tue-May-26-18:12:57-2015_qsofg.fits


In [237]:
# Collate Tables, calculate R_perp
qpq9 = copy.deepcopy(qso_bg)
# Add/replce key columns from qso_fg
oldc = ['Z_IR','SIGMA_ZIR', 'RA', 'DEC']
newc = ['FG_ZIR', 'FG_SIG_ZIR', 'FG_RA', 'FG_DEC']
for jj,ioldc in enumerate(oldc):
    tmpc = Column( qso_fg[ioldc], name=newc[jj])
    #print newc[jj]
    qpq9.add_column( tmpc )
# 
qpq9['G_UV'] = qso_fg['G_UV']
qpq9.add_column( Column( qso_bg['Z_TOT'], name='BG_Z') )
len(qpq9)

34

### Rphys

In [238]:
# Coordinates
c_fg = SkyCoord(ra=qpq9['FG_RA']*u.deg, dec=qpq9['FG_DEC']*u.deg)
c_bg = SkyCoord(ra=qpq9['RA']*u.deg, dec=qpq9['DEC']*u.deg)
# Separation
kpc_amin = cosmo.kpc_proper_per_arcmin( qpq9['FG_ZIR'] ) # kpc per arcmin
ang_sep = c_fg.separation(c_bg).to('arcmin')
Rphys = kpc_amin * ang_sep
# Add
if 'R_PHYS' not in qpq9.dtype.names:
    qpq9.add_column(Column( Rphys, name='R_PHYS'))
len(qpq9)

34

## Cuts

In [239]:
# Lya beta
zLya_beta = (qpq9['BG_Z']+1)*1025.7223 / 1215.6701 - 1
dv_beta = ltu.v_from_z(qpq9['FG_ZIR'], zLya_beta)
gd_beta = dv_beta > (-500*u.km/u.s)
#
qpq9 = qpq9[gd_beta]
print('We have {:d} pairs in QPQ9'.format(len(qpq9)))
#xdb.xpcol(zLya_beta, qpq9['FG_ZIR'], qpq9['BG_Z'])

We have 33 pairs in QPQ9


## Write

In [25]:
# Don't write. The current qpq9_zIR.fits made by Joe/X have more columns then what the code above gives. 
#xxf.table_to_fits(qpq9, 'qpq9_zIR.fits')

# list of NIR redshifts 

In [3]:
# Load masterlists
path = enigma.__path__[0]+'/data/redshifts/'
c_NIR = []
gnirs = Table.read(path+'gnirs_masterlist.dat',guess=False,format='ascii.csv',comment='#')
for qq in gnirs:
    c_NIR.append(ltu.radec_to_coord(qq['ra_qso']+' '+qq['dec_qso']))
niri = Table.read(path+'niri_masterlist.dat',guess=False,format='ascii.csv',comment='#')
for qq in niri:
    c_NIR.append(ltu.radec_to_coord(qq['ra_qso']+' '+qq['dec_qso']))
nirspec = Table.read(path+'nirspec_masterlist.dat',guess=False,format='ascii.csv',comment='#')
for qq in nirspec:
    c_NIR.append(ltu.radec_to_coord(qq['ra_qso']+' '+qq['dec_qso']))
xshooter = Table.read(path+'xshooter_masterlist.dat',guess=False,format='ascii.csv',comment='#')
for qq in xshooter:
    c_NIR.append(ltu.radec_to_coord(qq['ra_qso']+' '+qq['dec_qso']))
tspec = Table.read(path+'triplespec_masterlist.dat',guess=False,format='ascii.csv',comment='#')
for qq in tspec:
    c_NIR.append(ltu.radec_to_coord(qq['ra_qso']+' '+qq['dec_qso']))
isaac = Table.read(path+'isaac_masterlist.dat',guess=False,format='ascii.csv',comment='#')
for qq in isaac:
    c_NIR.append(ltu.radec_to_coord(qq['ra_qso']+' '+qq['dec_qso']))
lucifer = Table.read(path+'lucifer_masterlist.dat',guess=False,format='ascii.csv',comment='#')
for qq in lucifer:
    c_NIR.append(ltu.radec_to_coord(qq['ra_qso']+' '+qq['dec_qso']))    
liam = Table.read(path+'masterlist_liam.fits')
for qq in liam:
    c_NIR.append(SkyCoord(qq['RAJ2000']+qq['DEJ2000'],frame='fk5'))
c_NIR = SkyCoord(c_NIR)

# Load zem structures
zsys_line = ['H$\\alpha$','H$\\beta$','MgII','[OIII]']
zem_list = []
zem_coord = []
zem_fil = glob.glob(os.getenv('DROPBOX_DIR')+'Marie_Enrico/papers/QPQ9/Analysis/Redshifts/zem/From_X_Joe/*')
for ff in zem_fil:
    zem = xxf.bintab_to_table(ff)
    zem_list.append(zem)
    ipos = zem['SPEC_FIL'][0].rfind('/SDSSJ')+6
    radec = zem['SPEC_FIL'][0][ipos:]
    ipos2 = radec.rfind('+')
    if ipos2 == -1:
        ipos2 = radec.rfind('-')
    ipos3 = radec.rfind('_F.fits')
    if ipos3 == -1:
        ipos3 = radec.rfind('.fits')
    RA = radec[0:2]+'h'+radec[2:4]+'m'+radec[4:ipos2]+'s'
    dec = radec[ipos2:ipos2+3]+'d'+radec[ipos2+3:ipos2+5]+'m'+radec[ipos2+5:ipos3]+'s'
    zem_coord.append(SkyCoord(RA,dec))
zem_fil = glob.glob(os.getenv('DROPBOX_DIR')+'Marie_Enrico/papers/QPQ9/Analysis/Redshifts/zem/*.fits')
for ff in zem_fil:
    zem = xxf.bintab_to_table(ff)
    zem_list.append(zem)
    ipos = zem['SPEC_FIL'][0].rfind('/SDSSJ')+6
    radec = zem['SPEC_FIL'][0][ipos:]
    RA = radec[0:2]+'h'+radec[2:4]+'m'+radec[4:9]+'s'
    dec = radec[9:12]+'d'+radec[12:14]+'m'+radec[14:18]+'s'
    zem_coord.append(SkyCoord(RA,dec))
zem_coord = SkyCoord(zem_coord)

x.fits.bintab_to_table: Reading /Users/lwymarie/Dropbox/Marie_Enrico/papers/QPQ9/Analysis/Redshifts/zem/From_X_Joe/SDSSJ005717.36-000113.3_zem.fits
x.fits.bintab_to_table: Reading /Users/lwymarie/Dropbox/Marie_Enrico/papers/QPQ9/Analysis/Redshifts/zem/From_X_Joe/SDSSJ022517.68+004821.9_zem.fits
x.fits.bintab_to_table: Reading /Users/lwymarie/Dropbox/Marie_Enrico/papers/QPQ9/Analysis/Redshifts/zem/From_X_Joe/SDSSJ080049.89+354249.6_zem.fits
x.fits.bintab_to_table: Reading /Users/lwymarie/Dropbox/Marie_Enrico/papers/QPQ9/Analysis/Redshifts/zem/From_X_Joe/SDSSJ083757.91+383727.1_zem.fits
x.fits.bintab_to_table: Reading /Users/lwymarie/Dropbox/Marie_Enrico/papers/QPQ9/Analysis/Redshifts/zem/From_X_Joe/SDSSJ084158.47+392121.0_zem.fits
x.fits.bintab_to_table: Reading /Users/lwymarie/Dropbox/Marie_Enrico/papers/QPQ9/Analysis/Redshifts/zem/From_X_Joe/SDSSJ085358.36-001108.0_zem.fits
x.fits.bintab_to_table: Reading /Users/lwymarie/Dropbox/Marie_Enrico/papers/QPQ9/Analysis/Redshifts/zem/From_X_J

# Make a new sample merging QPQ9_zIR and QPQ7

In [4]:
zIR_fil = 'qpq9_zIR.fits'
QPQ7_fil = imp.find_module('enigma')[1] + '/data/qpq/qpq7_pairs.fits.gz'
QPQ_zIR = xxf.bintab_to_table(zIR_fil)
QPQ7 = xxf.bintab_to_table(QPQ7_fil)

x.fits.bintab_to_table: Reading qpq9_zIR.fits
x.fits.bintab_to_table: Reading /Users/lwymarie/python/enigma/enigma/data/qpq/qpq7_pairs.fits.gz


In [4]:
QPQ_zIR.keys()

['RUN',
 'RERUN',
 'CAMCOL',
 'FIELD',
 'ID',
 'PARENT',
 'NCHILD',
 'OBJC_TYPE',
 'OBJC_PROB_PSF',
 'OBJC_FLAGS',
 'OBJC_FLAGS2',
 'OBJC_ROWC',
 'OBJC_ROWCERR',
 'OBJC_COLC',
 'OBJC_COLCERR',
 'ROWV',
 'ROWVERR',
 'COLV',
 'COLVERR',
 'ROWC',
 'ROWCERR',
 'COLC',
 'COLCERR',
 'PSFCOUNTSERR',
 'FIBERCOUNTSERR',
 'FIBER2COUNTSERR',
 'PETROCOUNTSERR',
 'PETRORAD',
 'PETRORADERR',
 'PETROR50',
 'PETROR50ERR',
 'PETROR90',
 'PETROR90ERR',
 'Q',
 'QERR',
 'U',
 'UERR',
 'M_E1',
 'M_E2',
 'M_E1E1ERR',
 'M_E1E2ERR',
 'M_E2E2ERR',
 'M_RR_CC',
 'M_RR_CCERR',
 'M_CR4',
 'M_E1_PSF',
 'M_E2_PSF',
 'M_RR_CC_PSF',
 'M_CR4_PSF',
 'ISO_ROWC',
 'ISO_ROWCERR',
 'ISO_ROWCGRAD',
 'ISO_COLC',
 'ISO_COLCERR',
 'ISO_COLCGRAD',
 'ISO_A',
 'ISO_AERR',
 'ISO_AGRAD',
 'ISO_B',
 'ISO_BERR',
 'ISO_BGRAD',
 'ISO_PHIGRAD',
 'R_DEV',
 'R_DEVERR',
 'AB_DEV',
 'AB_DEVERR',
 'COUNTS_DEVERR',
 'R_EXP',
 'R_EXPERR',
 'AB_EXP',
 'AB_EXPERR',
 'COUNTS_EXPERR',
 'COUNTS_MODELERR',
 'STAR_LNL',
 'EXP_LNL',
 'DEV_LNL',
 'FRACP

In [5]:
QPQ9 = copy.deepcopy(QPQ_zIR)
QPQ9.rename_column('FG_ZIR', 'Z_FG')
QPQ9.rename_column('FG_SIG_ZIR','Z_FSIG')
for ii,qq in enumerate(QPQ9):
    try:
        assert 'J0225+0048' in qq['NAME'] # redshift is marked to be OII by mistake
        QPQ9.remove_row(ii)
    except AssertionError:
        pass
    try:
        assert 'J0913-0107' in qq['NAME'] # Hb redshift is not good, use MgII
        for zz,zem in enumerate(zem_list):
            if SkyCoord(qq['FG_RA']*u.deg,qq['FG_DEC']*u.deg).separation(zem_coord[zz]) < 5*u.arcsec:
                qq['Z_FG'] = zem['ZSYS_ZSYS'][0]
                qq['Z_FSIG'] = np.rint(zem['ZSYS_ERR'][0])
                qq['ZFG_LINE'] = zsys_line[np.min([zem['ZSYS_FLAG'][0],3])]   
    except AssertionError:
        pass
    try:
        assert 'J1433+0641' in qq['NAME'] # redshift is Null in the zIR structure 
        for zz,zem in enumerate(zem_list):
            if SkyCoord(qq['FG_RA']*u.deg,qq['FG_DEC']*u.deg).separation(zem_coord[zz]) < 5*u.arcsec:
                qq['Z_FG'] = zem['ZSYS_ZSYS'][0]
                qq['Z_FSIG'] = np.rint(zem['ZSYS_ERR'][0])
                qq['ZFG_LINE'] = zsys_line[np.min([zem['ZSYS_FLAG'][0],3])]
    except AssertionError:
        pass
len(QPQ9)

31

In [6]:
# Add QPQ7 pairs with NIR redshifts already measured 
c_QPQzIR = SkyCoord(ra=QPQ_zIR['FG_RA']*u.deg, dec=QPQ_zIR['FG_DEC']*u.deg)
c_QPQ7 = SkyCoord(ra=QPQ7['RAD']*u.deg, dec=QPQ7['DECD']*u.deg)
c_QPQ7_bg = SkyCoord(ra=QPQ7['RAD_BG']*u.deg,dec=QPQ7['DECD_BG']*u.deg)
kpc_amin = cosmo.kpc_proper_per_arcmin(QPQ7['Z_FG'])
ang_sep = c_QPQ7.separation(c_QPQ7_bg).to('arcmin')
                                          
for qq,cc in enumerate(c_QPQ7):
    # do not copy the entry with wrong spectral file 
    wrong_c = SkyCoord(ra=8.59753707028004*u.deg,dec=-10.832307685613417*u.deg)
    if c_QPQ7_bg[qq].separation(wrong_c).to('arcsec') < 0.5*u.arcsec:
        continue
    if len(np.where(cc.separation(c_QPQzIR).to('arcsec') < 0.5*u.arcsec)[0]) == 0:
        if kpc_amin[qq]*ang_sep[qq] < 300*u.kpc:
            if len(np.where(cc.separation(c_NIR) < 5*u.arcsec)[0]) > 0:
                print(cc.to_string('hmsdms'))
                for zz,zem in enumerate(zem_list):
                    if cc.separation(zem_coord[zz]) < 5*u.arcsec:
                        QPQ9.add_row({'FG_RA':QPQ7[qq]['RAD'],'FG_DEC':QPQ7[qq]['DECD'],'NAME':QPQ7[qq]['QSO_BG'],
                                      'RA':QPQ7[qq]['RAD_BG'],'DEC':QPQ7[qq]['DECD_BG'],
                                      'Z_FG':zem['ZSYS_ZSYS'][0],'Z_FSIG':np.rint(zem['ZSYS_ERR'][0]),
                                      'BG_Z':QPQ7[qq]['Z_BG'],'R_PHYS':QPQ7[qq]['R_PHYS'],
                                      'ZFG_LINE':zsys_line[np.min([zem['ZSYS_FLAG'][0],3])],
                                      'G_UV':QPQ7[qq]['G_UV']})
                        
QPQ9.sort(['FG_RA','FG_DEC'])

12h31m43.0985s +00d28m46.2929s


In [7]:
# Add QPQ7 pairs with MgII redshifts
c_QPQ9 = SkyCoord(ra=QPQ9['FG_RA']*u.deg, dec=QPQ9['FG_DEC']*u.deg)
for qq,cc in enumerate(c_QPQ7):
    # do not copy the entry with wrong spectral file 
    wrong_c = SkyCoord(ra=8.59753707028004*u.deg,dec=-10.832307685613417*u.deg)
    if c_QPQ7_bg[qq].separation(wrong_c).to('arcsec') < 0.5*u.arcsec:
        continue
    if len(np.where(cc.separation(c_QPQ9).to('arcsec') < 0.5*u.arcsec)[0]) == 0:
        if kpc_amin[qq]*ang_sep[qq] < 300*u.kpc:
            if QPQ7['Z_FSIG'][qq] < 300:
                QPQ9.add_row({'FG_RA':QPQ7['RAD'][qq],'FG_DEC':QPQ7['DECD'][qq],'NAME':QPQ7['QSO_BG'][qq],
                              'RA':QPQ7['RAD_BG'][qq],'DEC':QPQ7['DECD_BG'][qq],'Z_FG':QPQ7['Z_FG'][qq],'Z_FSIG':272,
                              'BG_Z':QPQ7['Z_BG'][qq],'R_PHYS':QPQ7['R_PHYS'][qq],'ZFG_LINE':'MgII',
                              'G_UV':QPQ7['G_UV'][qq]})

QPQ9.sort(['FG_RA','FG_DEC'])

In [8]:
len(QPQ9),QPQ9['RA','DEC','NAME','ZFG_LINE','BG_Z','Z_FG','G_UV'][0:20]

(56, <QTable length=20>
       RA             DEC             NAME      ...      Z_FG          G_UV    
    float64         float64          str14      ...    float64       float64   
 ------------- ----------------- -------------- ... ------------- -------------
 14.3287633684  -0.0263108702349 BOSSJ0057-0001 ... 2.16085586032 897.407834846
 27.3227662484   -0.366242429577 SDSSJ0149-0021 ... 1.68225049973 1726.12219238
 37.5832882888    -3.55444111362 BOSSJ0230-0333 ... 2.38024330139 1011.23883057
 55.4133096521 -0.00353495681141 SDSSJ0341-0000 ... 2.12328910828 274.194061279
 62.4758752949    -4.19363997107 SDSSJ0409-0411 ... 1.71549904346 515.653686523
 118.246405235     40.1884027001 SDSSJ0752+4011 ...  1.8831974268 749.980773926
  120.20306832     35.7086976546  APOJ0800+3542 ... 1.98231124769 1463.32499572
 127.186963574     45.7550664137 SDSSJ0828+4545 ... 1.87179803848 371.397277832
 129.302880827     36.5104726848 APOJ0837+3630  ...  1.8351764679 3560.37475586
 129.488047957  

In [8]:
# write
xxf.table_to_fits(QPQ9,'qpq9_final.fits')

# Check pairs in QPQ database and have NIR

In [9]:
# Find and add those pairs 
QPQ9 = xxf.bintab_to_table('qpq9_final.fits')
c_QPQ9 = SkyCoord(QPQ9['FG_RA']*u.deg,QPQ9['FG_DEC']*u.deg)
c_QPQ9_bg = SkyCoord(ra=QPQ9['RA']*u.deg,dec=QPQ9['DEC']*u.deg)
qpq_fil = '/Users/lwymarie/Desktop/qpq_optical.hdf5'
qpqsp = SpecDB(db_file=qpq_fil,verbose=False,idkey='PRIV_ID')
ID_fg, ID_bg = qpqsp.qcat.pairs(0.92*u.arcmin, 3000.*u.km/u.s)
c_qpqsp = SkyCoord(qpqsp.cat['RA'][ID_fg]*u.deg,qpqsp.cat['DEC'][ID_fg]*u.deg)
c_qpqsp_bg = SkyCoord(qpqsp.cat['RA'][ID_bg]*u.deg,qpqsp.cat['DEC'][ID_bg]*u.deg)
kpc_amin = cosmo.kpc_proper_per_arcmin(qpqsp.cat['zem'][ID_fg])
ang_sep = c_qpqsp.separation(c_qpqsp_bg).to('arcmin')
for qq,cc in enumerate(c_qpqsp):
    if kpc_amin[qq]*ang_sep[qq] < 300 *u.kpc: 
        if len(np.where(cc.separation(c_QPQ9) < 0.5*u.arcsec)[0]) == 0: # pair not in QPQ9 already
            if len(np.where(cc.separation(c_NIR) < 5*u.arcsec)[0]) > 0:
                print(cc.to_string('hmsdms'))
                #J0225 has no good line measured. Skip.
                if cc.separation(SkyCoord('02h25m17.68s +00d48m22s')) < 5*u.arcsec:
                    continue
                #J0239 add [OIII]. Did not pass Lyb cut. Add back. 
                #J1112 has good Hb, but currently it's MgII from QPQ7. 
                #The redshift from QPQ7 or Hb here does not pass the > 3000 km/s requirement. Skip. 
                if cc.separation(SkyCoord('11h12m42.69s +66d11m52.8s')) < 5*u.arcsec:
                    continue
                #J1215 add [OIII].
                #J2338 add [OIII]. 
                name = 'BOSSJ'
                name = name + c_qpqsp_bg[qq].to_string('hmsdms')[0:2] + c_qpqsp_bg[qq].to_string('hmsdms')[3:5]
                ipos = c_qpqsp_bg[qq].to_string('hmsdms').rfind(' ')+1
                name = name + c_qpqsp_bg[qq].to_string('hmsdms')[ipos:ipos+3] + c_qpqsp_bg[qq].to_string('hmsdms')[ipos+4:ipos+6]
                index = np.where(zem_coord.separation(cc) < 5*u.arcsec)[0]
                QPQ9.add_row({'FG_RA':qpqsp.cat[ID_fg[qq]]['RA'],'FG_DEC':qpqsp.cat[ID_fg[qq]]['DEC'],'NAME':name,
                              'RA':qpqsp.cat[ID_bg[qq]]['RA'],'DEC':qpqsp.cat[ID_bg[qq]]['DEC'],
                              'Z_FG':zem_list[index]['ZSYS_ZSYS'][0],'Z_FSIG':np.rint(zem_list[index]['ZSYS_ERR'][0]),
                                      'BG_Z':qpqsp.cat[ID_bg[qq]]['zem'],'R_PHYS':(kpc_amin[qq]*ang_sep[qq]).value,
                                      'ZFG_LINE':zsys_line[np.min([zem_list[index]['ZSYS_FLAG'][0],3])]})
QPQ9.sort(['FG_RA','FG_DEC'])

x.fits.bintab_to_table: Reading qpq9_final.fits
02h25m17.68s +00d48m22s
02h39m46.43s -01d06m40.4s
11h12m42.69s +66d11m52.8s
12h15m59s +57d16m16.6s
23h38m45.19s -00d03m27.2s


/Users/lwymarie/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:34: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
/Users/lwymarie/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:36: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future


In [10]:
# write
xxf.table_to_fits(QPQ9,'qpq9_final.fits')

# Find igmspec pairs with NIR spectra

In [11]:
# Find and add those pairs 
QPQ9 = xxf.bintab_to_table('qpq9_final.fits')
c_QPQ9 = SkyCoord(QPQ9['FG_RA']*u.deg,QPQ9['FG_DEC']*u.deg)
c_QPQ9_bg = SkyCoord(ra=QPQ9['RA']*u.deg,dec=QPQ9['DEC']*u.deg)
igmsp = IgmSpec()
ID_fg, ID_bg = igmsp.qcat.pairs(0.92*u.arcmin, 3000.*u.km/u.s) # at z = 0.4, 0.92' = 300 kpc
c_igmsp = SkyCoord(igmsp.cat['RA'][ID_fg]*u.deg,igmsp.cat['DEC'][ID_fg]*u.deg)
c_igmsp_bg = SkyCoord(igmsp.cat['RA'][ID_bg]*u.deg,igmsp.cat['DEC'][ID_bg]*u.deg)
kpc_amin = cosmo.kpc_proper_per_arcmin(igmsp.cat['zem'][ID_fg])
ang_sep = c_igmsp.separation(c_igmsp_bg).to('arcmin')

for qq,cc in enumerate(c_igmsp):
    if kpc_amin[qq]*ang_sep[qq] < 300 *u.kpc: 
        if len(np.where(cc.separation(c_QPQ9) < 0.5*u.arcsec)[0]) == 0: # pair not in QPQ9 already
            if len(np.where(cc.separation(c_NIR) < 5*u.arcsec)[0]) > 0:
                print(cc.to_string('hmsdms'))         
                # Skip J0225. No good line measured. 
                if cc.separation(SkyCoord('02h25m17.68s +00d48m22s')) < 5*u.arcsec:
                    continue
                # Skip J0201. Binary quasar, with wrong redshift in catalog. 
                if cc.separation(SkyCoord('02h01m43.4873s +00d32m22.713s')) < 5*u.arcsec:
                    continue
                name = 'SDSSJ'
                name = name + c_igmsp_bg[qq].to_string('hmsdms')[0:2] + c_igmsp_bg[qq].to_string('hmsdms')[3:5]
                ipos = c_igmsp_bg[qq].to_string('hmsdms').rfind(' ')+1
                name = name + c_igmsp_bg[qq].to_string('hmsdms')[ipos:ipos+3] + c_igmsp_bg[qq].to_string('hmsdms')[ipos+4:ipos+6]
                index = np.where(zem_coord.separation(cc) < 5*u.arcsec)[0]
                QPQ9.add_row({'FG_RA':igmsp.cat[ID_fg[qq]]['RA'],'FG_DEC':igmsp.cat[ID_fg[qq]]['DEC'],'NAME':name,
                              'RA':igmsp.cat[ID_bg[qq]]['RA'],'DEC':igmsp.cat[ID_bg[qq]]['DEC'],
                              'Z_FG':zem_list[index]['ZSYS_ZSYS'][0],'Z_FSIG':np.rint(zem_list[index]['ZSYS_ERR'][0]),
                                      'BG_Z':igmsp.cat[ID_bg[qq]]['zem'],'R_PHYS':(kpc_amin[qq]*ang_sep[qq]).value,
                                      'ZFG_LINE':zsys_line[np.min([zem_list[index]['ZSYS_FLAG'][0],3])]})
QPQ9.sort(['FG_RA','FG_DEC'])
                

x.fits.bintab_to_table: Reading qpq9_final.fits
02h25m17.6775s +00d48m21.9398s
02h01m43.4873s +00d32m22.713s
01h11m50.07s +14d01m41.5s


/Users/lwymarie/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:30: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
/Users/lwymarie/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:32: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future


In [13]:
len(QPQ9),QPQ9['ZFG_LINE','NAME'][0:20]

(60, <QTable length=20>
  ZFG_LINE      NAME     
   str36       str14     
 --------- --------------
    [OIII] BOSSJ0057-0001
  H$\beta$ SDSSJ0111+1402
      MgII SDSSJ0149-0021
      MgII BOSSJ0230-0333
    [OIII] BOSSJ0239-0106
      MgII SDSSJ0341-0000
      MgII SDSSJ0409-0411
      MgII SDSSJ0752+4011
    [OIII]  APOJ0800+3542
      MgII SDSSJ0828+4545
      MgII  APOJ0837+3630
 H$\alpha$  APOJ0837+3837
    [OIII] BOSSJ0841+3921
    [OIII] BOSSJ0853-0011
    [OIII]  APOJ0910+0414
      MgII BOSSJ0913-0107
    [OIII] BOSSJ0914+0109
      MgII  APOJ0924+3929
    [OIII]  APOJ0932+0925
      MgII BOSSJ0936-0058)

In [12]:
# write
xxf.table_to_fits(QPQ9,'qpq9_final.fits')

# Add igmspec pairs with HW10 redshifts 

In [13]:
for qq,cc in enumerate(c_igmsp):
    if kpc_amin[qq]*ang_sep[qq] < 300 *u.kpc: 
        if len(np.where(cc.separation(c_QPQ9) < 0.5*u.arcsec)[0]) == 0: # pair not in QPQ9 already
            if len(np.where(cc.separation(c_NIR) < 0.5*u.arcsec)[0]) == 0:                 
                if 'SDSS-HW' in igmsp.cat[ID_fg[qq]]['flag_zem']: # good z
                    # Match Johnson+15 and Farina+ samples' redshifts, and MgII redshift or better 
                    if (igmsp.cat[ID_fg[qq]]['zem'] >= 0.4) & (igmsp.cat[ID_fg[qq]]['zem'] <= 2.1): 
                        # Lya forest cut for MgII stack 
                        if 2796.354*(1+igmsp.cat['zem'][ID_fg[qq]]) > (1215.6701+6.)*(1+igmsp.cat['zem'][ID_bg[qq]]):
                            print(cc.to_string('hmsdms')) 
                            if np.mod(igmsp.cat['flag_group'][ID_fg[qq]],2) == 0:
                                name = 'BOSSJ'
                            else:
                                name = 'SDSSJ'  
                            name = name + c_igmsp_bg[qq].to_string('hmsdms')[0:2]
                            name = name + c_igmsp_bg[qq].to_string('hmsdms')[3:5]
                            ipos = c_igmsp_bg[qq].to_string('hmsdms').rfind(' ')+1
                            name = name + c_igmsp_bg[qq].to_string('hmsdms')[ipos:ipos+3] 
                            name = name + c_igmsp_bg[qq].to_string('hmsdms')[ipos+4:ipos+6]
                            QPQ9.add_row({'FG_RA':igmsp.cat['RA'][ID_fg[qq]],'FG_DEC':igmsp.cat['DEC'][ID_fg[qq]],
                                          'NAME':name,
                                          'RA':igmsp.cat['RA'][ID_bg[qq]],'DEC':igmsp.cat['DEC'][ID_bg[qq]],
                                          'Z_FG':igmsp.cat['zem'][ID_fg[qq]],'Z_FSIG':272,
                                          'BG_Z':igmsp.cat['zem'][ID_bg[qq]],
                                          'R_PHYS':(kpc_amin[qq]*ang_sep[qq]).value,'ZFG_LINE':'SDSS-HW'})
QPQ9.sort(['FG_RA','FG_DEC'])

00h01m25.1443s +00d00m09.4356s
00h03m16.3195s -00d21m48.9276s
00h08m04.2269s -00d39m22.9824s
00h08m05.1814s -00d39m46.782s
00h12m57.2542s +01d15m27.342s
00h21m49.9378s +16d02m08.718s
00h24m44.1209s +00d32m21.2136s
00h25m49.3752s -00d54m20.592s
00h26m23.7881s +13d55m23.538s
00h29m28.8019s +00d02m08.1744s
00h30m33.258s +00d55m06.042s
00h33m49.8674s +15d46m13.3104s
00h36m44.718s +16d08m30.6996s
00h39m05.9023s -00d07m01.6824s
00h40m39.719s +14d08m27.4524s
00h41m26.6724s +01d14m37.2696s
00h41m36.325s +13d55m33.294s
00h42m38.2385s +13d50m54.8052s
00h43m38.2918s +00d05m23.8056s
00h44m42.0742s -00d42m10.296s
00h45m54.642s +00d11m11.8212s
00h47m57.2575s +14d47m41.9604s
00h56m14.8121s +01d03m33.0336s
00h57m19.7537s -01d00m10.4112s
00h58m28.9363s +00d13m46.8696s
01h00m07.2816s -00d32m18.5496s
01h03m23.8409s -00d02m54.204s
01h03m48.1382s +15d01m57.3312s
01h06m16.3968s +00d56m56.85s
01h09m28.3505s +01d04m32.034s
01h10m34.7911s +00d02m30.4656s
01h12m39.6792s +00d19m43.4712s
01h14m30.2426s +00d04m20.

In [18]:
len(QPQ9),QPQ9['NAME','ZFG_LINE','Z_FG'][0:20]

(654, <QTable length=20>
      NAME      ZFG_LINE      Z_FG     
     str14       str36      float64    
 -------------- -------- --------------
 BOSSJ0001+0000  SDSS-HW  1.98956203461
 BOSSJ0003-0022  SDSS-HW 0.519200026989
 BOSSJ0008-0039  SDSS-HW  1.61289596558
 BOSSJ0008-0039  SDSS-HW  1.93808495998
 BOSSJ0012+0115  SDSS-HW 0.504666984081
 BOSSJ0021+1602  SDSS-HW 0.836601018906
 BOSSJ0024+0032  SDSS-HW 0.402581989765
 BOSSJ0025-0054  SDSS-HW 0.561591982841
 BOSSJ0026+1354  SDSS-HW  1.31915104389
 BOSSJ0029+0001  SDSS-HW 0.980237007141
 BOSSJ0030+0054  SDSS-HW  1.45808899403
 BOSSJ0033+1546  SDSS-HW  1.26510596275
 BOSSJ0036+1609  SDSS-HW  0.46544200182
 BOSSJ0039-0007  SDSS-HW  1.01221096516
 BOSSJ0040+1409  SDSS-HW 0.672659993172
 BOSSJ0041+0115  SDSS-HW  1.19738805294
 BOSSJ0041+1355  SDSS-HW 0.882438004017
 BOSSJ0042+1350  SDSS-HW  1.77609395981
 BOSSJ0043+0005  SDSS-HW  0.82183098793
 BOSSJ0044-0042  SDSS-HW 0.558131992817)

In [ ]:
# write
xxf.table_to_fits(QPQ9,'qpq9_final.fits')

In [ ]:
# Find pairs in QPQ database that have [OIII] at accessible wavelengths, but not in QPQ9 
QPQ9 = xxf.bintab_to_table('qpq9_final.fits')
c_QPQ9 = SkyCoord(QPQ9['FG_RA']*u.deg,QPQ9['FG_DEC']*u.deg)
c_QPQ9_bg = SkyCoord(ra=QPQ9['RA']*u.deg,dec=QPQ9['DEC']*u.deg)
qpq_fil = '/Users/lwymarie/Desktop/qpq_optical.hdf5'
qpqsp = SpecDB(db_file=qpq_fil,verbose=False,idkey='PRIV_ID')
ID_fg, ID_bg = qpqsp.qcat.pairs(0.92*u.arcmin, 3000.*u.km/u.s)
c_qpqsp = SkyCoord(qpqsp.cat['RA'][ID_fg]*u.deg,qpqsp.cat['DEC'][ID_fg]*u.deg)
c_qpqsp_bg = SkyCoord(qpqsp.cat['RA'][ID_bg]*u.deg,qpqsp.cat['DEC'][ID_bg]*u.deg)
kpc_amin = cosmo.kpc_proper_per_arcmin(qpqsp.cat['zem'][ID_fg])
ang_sep = c_qpqsp.separation(c_qpqsp_bg).to('arcmin')
for qq,cc in enumerate(c_qpqsp):
    if kpc_amin[qq]*ang_sep[qq] < 300 *u.kpc: 
        if len(np.where(cc.separation(c_QPQ9) < 0.5*u.arcsec)[0]) == 0: # pair not in QPQ9 already
                if (((qpqsp.cat['zem'][ID_fg[qq]] > 1.6) & (qpqsp.cat['zem'][ID_fg[qq]] < 1.67)) | 
                    ((qpqsp.cat['zem'][ID_fg[qq]] > 1.99) & (qpqsp.cat['zem'][ID_fg[qq]] < 2.60)) | 
                    ((qpqsp.cat['zem'][ID_fg[qq]] > 3.08) & (qpqsp.cat['zem'][ID_fg[qq]] < 3.76))):
                    print(qpqsp.cat['zem'][ID_fg[qq]],cc.to_string('hmsdms'),c_qpqsp_bg[qq].to_string('hmsdms'))

x.fits.bintab_to_table: Reading qpq9_final.fits
(2.4860000610351562, u'15h29m29.03s +23d14m20s', u'15h29m28.38s +23d14m15.9s')
(3.2240000000000002, u'08h51m59.89s +16d55m34.6s', u'08h51m59.07s +16d55m32.5s')
(2.3959717750549316, u'10h39m31.8s +18d44m56.8s', u'10h39m31.72s +18d44m51.7s')
(2.5510000000000002, u'11h49m08.77s +03d46m13.2s', u'11h49m09.77s +03d46m16.8s')
(2.5120000839233398, u'13h43m42.1s +00d19m20.8s', u'13h43m43.03s +00d19m02.4s')
(2.5630000000000002, u'22h51m42.1s +01d37m02.4s', u'22h51m41.39s +01d36m54s')
(1.6200000000000001, u'10h19m47.1s +49d48m35.8s', u'10h19m47s +49d48m49.1s')

In [ ]:
# Try 13h44m41.1s -00d49m50.2s with TSpec 

In [ ]:
# Find pairs in igmspec database that have [OIII] at accessible wavelengths, but not in QPQ9 
igmsp = IgmSpec()
ID_fg, ID_bg = igmsp.qcat.pairs(0.92*u.arcmin, 3000.*u.km/u.s)
c_igmsp = SkyCoord(igmsp.cat['RA'][ID_fg]*u.deg,igmsp.cat['DEC'][ID_fg]*u.deg)
c_igmsp_bg = SkyCoord(igmsp.cat['RA'][ID_bg]*u.deg,igmsp.cat['DEC'][ID_bg]*u.deg)
kpc_amin = cosmo.kpc_proper_per_arcmin(igmsp.cat['zem'][ID_fg])
ang_sep = c_igmsp.separation(c_igmsp_bg).to('arcmin')
for qq,cc in enumerate(c_igmsp):
    if kpc_amin[qq]*ang_sep[qq] < 300 *u.kpc: 
        if len(np.where(cc.separation(c_QPQ9) < 0.5*u.arcsec)[0]) == 0: # pair not in QPQ9 already
                if (((igmsp.cat['zem'][ID_fg[qq]] > 1.6) & (igmsp.cat['zem'][ID_fg[qq]] < 1.67)) | 
                    ((igmsp.cat['zem'][ID_fg[qq]] > 1.99) & (igmsp.cat['zem'][ID_fg[qq]] < 2.60)) | 
                    ((igmsp.cat['zem'][ID_fg[qq]] > 3.08) & (igmsp.cat['zem'][ID_fg[qq]] < 3.76))):
                    print(igmsp.cat['zem'][ID_fg[qq]],cc.to_string('hmsdms'),c_igmsp_bg[qq].to_string('hmsdms'))

# Add and update g_UV values 

In [27]:
# Search SDSS DR12 catalog for magnitudes 
QPQ9 = xxf.bintab_to_table('qpq9_final.fits')
c_QPQ9 = SkyCoord(QPQ9['FG_RA']*u.deg,QPQ9['FG_DEC']*u.deg)
c_QPQ9_bg = SkyCoord(ra=QPQ9['RA']*u.deg,dec=QPQ9['DEC']*u.deg)
hdulist = fits.open('/Users/lwymarie/Desktop/DR12Q.fits')
SDSSdr12 = hdulist[1].data
c_SDSSdr12 = SkyCoord(list(SDSSdr12.field('RA'))*u.deg,list(SDSSdr12.field('DEC'))*u.deg)
hdulist = fits.open('/Users/lwymarie/Desktop/dr7qso.fit')
SDSSdr7 = hdulist[1].data
c_SDSSdr7 = SkyCoord(list(SDSSdr7.field('RA'))*u.deg,list(SDSSdr7.field('DEC'))*u.deg)

x.fits.bintab_to_table: Reading qpq9_final.fits


In [28]:
# Put in g_UV by hand for a quasar that does not have SDSS spectra but photometry and no existing g_UV values 
for qq,cc in enumerate(c_QPQ9):
    if QPQ9[qq]['G_UV'] == 0.:
        if len(np.where(cc.separation(c_SDSSdr12) < 0.5*u.arcsec)[0]) == 0:
            if len(np.where(cc.separation(c_SDSSdr7) < 0.5*u.arcsec)[0]) == 0:
                ang_sep = cc.separation(c_QPQ9_bg[qq]).to('arcsec')
                print(cc.to_string('hmsdms'),QPQ9[qq]['Z_FG'],ang_sep,qq)
                QPQ9[qq]['G_UV'] = 4624

(u'01h11m50.07s +14d01m41.5s', 2.4851422501937015, <Angle 35.64055373774893 arcsec>, 32)


In [29]:
# Write a file for IDL input, for rescaling QPQ g_UV values to F.-G. Lya forest flux 
gUV_exist = np.where(QPQ9['G_UV'] != 0.)[0]
data = Table([QPQ9[gUV_exist]['FG_RA'],QPQ9[gUV_exist]['FG_DEC'],QPQ9[gUV_exist]['Z_FG'],QPQ9[gUV_exist]['G_UV']],
             names=['FG_RA','FG_DEC','Z_FG','OLD_GUV'])
ascii.write(data,'QPQ_gUV_idl_input.txt')

# Write a file for IDL input, for calculating new g_UV values 
gUV_none = np.where(QPQ9['G_UV'] == 0.)[0]
ang_sep = []
SDSSfilter = []
mag = []
filter_beg = [3125.0, 3880.0, 5480.0, 6790.0, 8090.0]
filter_names = ['u','g','r','i','z']
for qq,cc in enumerate(c_QPQ9[gUV_none]):
    ang_sep.append(cc.separation(c_QPQ9_bg[gUV_none[qq]]).to('arcsec').value)
    min_beg = 1215.6701*(1+QPQ9[gUV_none[qq]]['Z_FG'])
    good_inds = np.where(filter_beg >= min_beg)
    SDSSfilter.append(filter_names[np.min(good_inds)])
    in_dr12 = np.where(cc.separation(c_SDSSdr12) < 0.5*u.arcsec)[0]
    if len(in_dr12) == 1:
        mag.append((SDSSdr12[in_dr12].field('PSFMAG')[0][good_inds]-
                    SDSSdr12[in_dr12].field('EXTINCTION')[0][good_inds])[0])
    else:
        in_dr7 = np.where(cc.separation(c_SDSSdr7) < 0.5*u.arcsec)[0]
        psfmag = [SDSSdr7[in_dr7].field('UMAG'),SDSSdr7[in_dr7].field('GMAG'),SDSSdr7[in_dr7].field('RMAG'),
                  SDSSdr7[in_dr7].field('IMAG'),SDSSdr7[in_dr7].field('ZMAG')]
        extinct = SDSSdr7[in_dr7].field('AU')*np.array([1,0.736,0.534,0.405,0.287])
        mag.append((psfmag[np.min(good_inds)]-extinct[np.min(good_inds)])[0])
data = Table([QPQ9[gUV_none]['FG_RA'],QPQ9[gUV_none]['FG_DEC'],QPQ9[gUV_none]['Z_FG'],ang_sep,SDSSfilter,mag],
             names=['FG_RA','FG_DEC','Z_FG','theta','filter','mag'])
ascii.write(data,'no_gUV_idl_input.txt')

In [30]:
# Read IDL output for rescaled QPQ g_UV values 
data = ascii.read('QPQ_gUV_idl_output.txt')
c_idl = SkyCoord(data['FG_RA']*u.deg,data['FG_DEC']*u.deg)
for qq,cc in enumerate(c_QPQ9):
    index = np.where(cc.separation(c_idl) < 0.5*u.arcsec)[0]
    if len(index) == 1:
        QPQ9[qq]['G_UV'] = data[index]['NEW_GUV']
# Read IDL output for new g_UV values
data = ascii.read('no_gUV_idl_output.txt')
c_idl = SkyCoord(data['FG_RA']*u.deg,data['FG_DEC']*u.deg)
for qq,cc in enumerate(c_QPQ9):
    index = np.where(cc.separation(c_idl) < 0.5*u.arcsec)[0]
    if len(index) == 1:
        QPQ9[qq]['G_UV'] = data[index]['GUV']

In [31]:
# write
xxf.table_to_fits(QPQ9,'qpq9_final.fits')

# Update systematic offsets and scatters of emission lines to Shen+16 

In [33]:
# Add the systematic biases, intrinsic scatter + measurement scatter
QPQ9 = xxf.bintab_to_table('qpq9_final.fits')
for ii,qq in enumerate(QPQ9):
    if qq['ZFG_LINE'] == 'MgII':
        # bring to z_em 
        qq['Z_FG'] = ltu.z_from_v(qq['Z_FG'],+70.)
        # bring to z_sys
        qq['Z_FG'] = ltu.z_from_v(qq['Z_FG'],+57.)
        # scatter
        qq['Z_FSIG'] = 226
    if qq['ZFG_LINE'] == 'H$\\beta$':
        # bring to z_sys
        qq['Z_FG'] = ltu.z_from_v(qq['Z_FG'],+109.)
        # scatter 
        qq['Z_FSIG'] = 417
    if qq['ZFG_LINE'] == '[OIII]':
        # bring to z_em 
        qq['Z_FG'] = ltu.z_from_v(qq['Z_FG'],-27.)
        # bring to z_sys
        qq['Z_FG'] == ltu.z_from_v(qq['Z_FG'],+48.)
        # scatter
        qq['Z_FSIG'] = 68
    if qq['ZFG_LINE'] == 'H$\\alpha$':
        # scatter
        qq['Z_FSIG'] = 300
    if qq['ZFG_LINE'] == 'SDSS-HW':
        if qq['Z_FG'] >= 0.84:
            # MgII redshift, bring to z_sys
            qq['Z_FG'] = ltu.z_from_v(qq['Z_FG'],+57.)
            # scatter
            qq['Z_FSIG'] = 226
        else:
            # [OIII] redshift, bring to zsys
            qq['Z_FG'] = ltu.z_from_v(qq['Z_FG'],+48.)
            # scatter
            qq['Z_FSIG'] = 68

x.fits.bintab_to_table: Reading qpq9_final.fits


In [36]:
# write
xxf.table_to_fits(QPQ9,'qpq9_final.fits')

# copy to enigma

In [37]:
!cp qpq9_final.fits /Users/lwymarie/python/enigma/enigma/data/qpq/qpq9_final.fits
len(QPQ9)

654

In [5]:
QPQ9 = xxf.bintab_to_table('qpq9_final.fits')

x.fits.bintab_to_table: Reading qpq9_final.fits


In [6]:
QPQ9.keys()

['RUN',
 'RERUN',
 'CAMCOL',
 'FIELD',
 'ID',
 'PARENT',
 'NCHILD',
 'OBJC_TYPE',
 'OBJC_PROB_PSF',
 'OBJC_FLAGS',
 'OBJC_FLAGS2',
 'OBJC_ROWC',
 'OBJC_ROWCERR',
 'OBJC_COLC',
 'OBJC_COLCERR',
 'ROWV',
 'ROWVERR',
 'COLV',
 'COLVERR',
 'ROWC',
 'ROWCERR',
 'COLC',
 'COLCERR',
 'PSFCOUNTSERR',
 'FIBERCOUNTSERR',
 'FIBER2COUNTSERR',
 'PETROCOUNTSERR',
 'PETRORAD',
 'PETRORADERR',
 'PETROR50',
 'PETROR50ERR',
 'PETROR90',
 'PETROR90ERR',
 'Q',
 'QERR',
 'U',
 'UERR',
 'M_E1',
 'M_E2',
 'M_E1E1ERR',
 'M_E1E2ERR',
 'M_E2E2ERR',
 'M_RR_CC',
 'M_RR_CCERR',
 'M_CR4',
 'M_E1_PSF',
 'M_E2_PSF',
 'M_RR_CC_PSF',
 'M_CR4_PSF',
 'ISO_ROWC',
 'ISO_ROWCERR',
 'ISO_ROWCGRAD',
 'ISO_COLC',
 'ISO_COLCERR',
 'ISO_COLCGRAD',
 'ISO_A',
 'ISO_AERR',
 'ISO_AGRAD',
 'ISO_B',
 'ISO_BERR',
 'ISO_BGRAD',
 'ISO_PHIGRAD',
 'R_DEV',
 'R_DEVERR',
 'AB_DEV',
 'AB_DEVERR',
 'COUNTS_DEVERR',
 'R_EXP',
 'R_EXPERR',
 'AB_EXP',
 'AB_EXPERR',
 'COUNTS_EXPERR',
 'COUNTS_MODELERR',
 'STAR_LNL',
 'EXP_LNL',
 'DEV_LNL',
 'FRACP